# Ag-Analytics-Harmonised Landsat Sentinel API-Demo


<br>The Harmonized Landsat and Sentinel-2 (HLS) is a NASA initiative to produce a Virtual Constellation (VC) of surface reflectance (SR) data from the Operational Land Imager (OLI) and MultiSpectral Instrument (MSI) onboard the Landsat 8 and Sentinel-2 remote sensing satellites, respectively. The data from these satellites creates unprecedented opportunities for timely and accurate observation of Earth status and dynamics at moderate (<30 m) spatial resolution every 2-3 days.

<br>The Ag-Analytics provides the service which a user can provide a polygon in specific formats in additional with more customized options to retrieve the dynamics of their land at various times for Landsat from 2013 and Sentinel-2 from 2015 calendar years. Additionally, this service also provides information on cloud cover, statistics, Normalized Difference Vegetation Index in addition to MSI bands information.

<br>

<br> Experience the usage of this service on Ag-Analytics website using below link
<br>https://ag-analytics.portal.azure-api.net/docs/services/harmonized-landsat-sentinel-service/operations/hls-service

In [1]:
import requests
import json
import time
import os
from pandas.io.json import json_normalize
from collections import defaultdict
import pandas as pd
import zipfile, io
from IPython.display import Image

%matplotlib inline
%autosave 0

Autosave disabled


###### Request parameters details

Request URL: https://aganalyticsapimanagementservice.azure-api.net/hls/detect_and_clip_tiles


1) aoi (geometry, file/text, required): The structure of the geometry can be one of the following.
<br>
i.	  JSON geometry objects returned by the arcgis rest api, (file/text) 
ii.	  GEOJSON (file/text)
iii.  Shapefile (file)
iv.	  Raster of tiff extension (file)


2) Band (Spectral band name, list, required): Provide the list of HLS Spectral band names to retrieve for given aoi<br>
•	Currently Supporting: Red, Green, Blue, NDVI, Coastal Aerosol, NIR


3) satellite (Type of satellite, text, optional):<br>  Landsat or Sentinel. Default (Landsat)

4) resolution (cellsize in meters, float, optional):<br> Default (0.0001)


5) Startdate (Date, mm/dd/yyyy, optional): Enter the starting date to capture the details from Landsat/Sentinel-2<br>
•	Year in Startdate for Landsat should start from 2013 <br>
•	Year in Startdate for Sentinel should start from 2015 <br>
•	Note Startdate and Enddate both should be given, In the absence of other the service retrieves the latest information available on the land.<br>

6) Enddate (Date, mm/dd/yyyy, optional): Enter the end date to capture the details from Landsat/Sentinel-2<br>
•	Year in Enddate for Landsat can be till current year <br>
•	Year in Enddate for Sentinel can be till current year <br>
•	Note Startdate and Enddate both should be given, In the absence of other the service retrieves the latest information available on the land<br>

7) byweek (int,boolean,optional):This parameter will mosaic the rasters in a week Default 1


In order to get Ocp-Apim-Subscription-Key, please click on this link https://ag-analytics.org/developer/Session/SignInFromPayment

### Request Parameters

In [2]:
values={"aoi": '{"type":"Feature","geometry":{"type":"Polygon","coordinates":[[[-93.511545,42.071053],[-93.511565,42.074566],[-93.50667,42.074588],[-93.501908,42.074559],[-93.501936,42.071045],[-93.511545,42.071053]]]},"properties":{"OBJECTID":3350330,"CALCACRES":77.09999847,"CALCACRES2":null},"id":3350330}',
"Band": "['NDVI']",
"Enddate": "3/8/2019",
"Startdate": "3/2/2019",
"legendtype": "Relative",
"satellite": "Landsat,Sentinel",
"resolution":"0.00001",
"byweek":"1"}


    
headers={'Content-Type':'application/x-www-form-urlencoded','Ocp-Apim-Subscription-Key': 'xxxxxxxxxxxxxxxxxxxxxxxxxxxxxx'}
           

###  API Function

In [5]:
def harmonisedlandsatdata(values,headers):
    try:
        url = "https://aganalyticsapimanagementservice.azure-api.net/detect_and_clip_tiles"        
     
        response = requests.post(url, data=values,headers=headers).json()
        
        print(response)

        return response
    
    except Exception as e:
        print(e)
        raise e

### Calling API Function and Displaying Response

In [6]:
hlsresponse=harmonisedlandsatdata(values,headers)

[{'tiledate': '02/25/2019-03/03/2019', 'band': 'NDVI', 'download_url': 'downloads/raster_bandNDVI_date2019056-2019062_20191204_205712_3537.tif', 'features': [{'attributes': {'OID': 0, 'Extent': '-93.51155513090414, 42.071050716073884, -93.50191513090414, 42.07458071607388', 'Mean': -0.05406530029859541, 'Max': -0.05220879137841957, 'Min': -0.05630117657168601, 'Std': 0.0006640823555605791, 'Percentile5': -0.05510107010083765, 'Percentile95': -0.052937400036051505, 'pngb64': 'data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAA8QAAAFhCAYAAAC24mODAAAyTElEQVR4nO3dWXYbSbIFwGCdt1ZyTeRm9T6q0AJJjIkY3D3M+uinSyITMfrNSACtAQAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAPTytvoCVnv/8/7n1n//bF+zLuWg9xv/bc61f9y8hjni99M4Edq/uqjjS9//9VofDWjHjw4/47PDzxgu5tz46d5ciTrHZ4i8jnwl6pf3wO04Sqb+uWZGv81ZXwa+jkf3syV7Vp+2/afLT2GhawMh/yIFkNarhUGKMAzALdvcbEu+ZwnEJW0y+QAiO1IgfB78d9y0TVEKE2U/Fc9+/fQjEJfw9

### Displaying  Output Image

In [7]:
for resp in hlsresponse:
    if resp["band"]=="NDVI" and resp["download_url"]!="":
        
        print("Legend Values : ",resp["features"][0]['attributes']['Legend'])
        
        img_uri=resp["features"][0]['attributes']["pngb64"]
        
        Image(url=img_uri)

        
        

Image(url=img_uri)        
        

Legend Values :  [{'color': '#ff0000', 'Min': -0.05630117657168601, 'Max': -0.05561911237280827, 'Mean': -0.05596014447224714, 'Area': '0.6 %', 'Count': 1702, 'CountAllPixels': 281440}, {'color': '#ff6666', 'Min': -0.05561911237280827, 'Max': -0.05493704817393053, 'Mean': -0.055278080273369404, 'Area': '9.51 %', 'Count': 26764, 'CountAllPixels': 281440}, {'color': '#ffff66', 'Min': -0.05493704817393053, 'Max': -0.054254983975052795, 'Mean': -0.05459601607449166, 'Area': '30.2 %', 'Count': 84990, 'CountAllPixels': 281440}, {'color': '#ffff00', 'Min': -0.054254983975052795, 'Max': -0.05357291977617505, 'Mean': -0.05391395187561392, 'Area': '35.27 %', 'Count': 99263, 'CountAllPixels': 281440}, {'color': '#66ff66', 'Min': -0.05357291977617505, 'Max': -0.05289085557729731, 'Mean': -0.05323188767673618, 'Area': '19.93 %', 'Count': 56104, 'CountAllPixels': 281440}, {'color': '#00ff00', 'Min': -0.05289085557729731, 'Max': -0.05220879137841957, 'Mean': -0.05254982347785844, 'Area': '4.48 %', 'C

# GET Request

GET request for downloading the HLS images returned in the POST request.

In [8]:
# User specified local path where file will be downloaded.
local_path = r"C:\<Path to download directory>" # E.g., r"C:\Users\John_Doe\Documents\rasters"

In [19]:
def HLS_get(local_path):
    try:
        url = "https://aganalyticsapimanagementservice.azure-api.net/detect_and_clip_tiles"
    
        for resp in hlsresponse:
            file_url = resp['download_url']
            file_name = file_url.split('/')[-1]
            download_path = os.path.join(local_path, file_name)
            
            values = {'filename': file_name}
     
            response = requests.get(url, params=values)
            open(download_path, 'wb').write(response.content)
        
            print(response.url)

        return response
    
    except Exception as e:
        print(e)
        raise e

In [20]:
HLS_get(local_path)

https://agflaskdev.eastus2.cloudapp.azure.com/flask_api/api/v1.0/tile_service/detect_and_clip_tiles?filename=raster_bandNDVI_date2019056-2019062_20191204_205712_3537.tif
https://agflaskdev.eastus2.cloudapp.azure.com/flask_api/api/v1.0/tile_service/detect_and_clip_tiles?filename=raster_bandNDVI_date2019063-2019069_20191204_205713_8814.tif


<Response [200]>